In [ ]:
#install stuff
# IMPORTANT NOTE: before starting Jupyter Notebook, navigate to /home/peter/Desktop/yolov5. Do source .venv/bin/activate
# this sets up the virtual environment where torch is installed and allows yolo to work.
# if installing this fresh, you'll need to install torch with Cuda (or not depending on the hardware)
#Yolov5
%cd /home/peter/Desktop/yolov5
!source .venv/bin/activate
#pip install python>=3.1.30
!pip install matplotlib>=3.3
!pip install numpy>=1.23.5
!pip install opencv-python>=4.1.1
!pip install pillow>=10.3.0
!pip install psutil  # system resources
!pip install PyYAML>=5.3.1
!pip install requests>=2.32.2
!pip install scipy>=1.4.1
!pip install hop>=0.1.1  # FLOPs computation
!pip install torch>=1.8.0  # see https://pytorch.org/get-started/locally (recommended)
!pip install torchvision>=0.9.0
!pip install tqdm>=4.66.3
!pip install ultralytics>=8.2.64  # https://ultralytics.com
!pip install pandas>=1.1.4
!pip install seaborn>=0.11.0
# assume the files are in the right places. See https://github.com/ch-hristov/p-id-symbols for details. Follow the notebook
#Ollama
!pip install ollama
#Claude
!pip install anthropic


In [116]:
#yolo
import pandas as pd
import numpy as np
from PIL import Image
import ast
import os
from pathlib import Path
import shutil

import argparse
import csv
import platform
import sys
import torch

#ollama
import ollama
import base64
# import os duplicate
import glob
from pathlib import Path
import json
from typing import List, Dict, Optional

#Claude
import anthropic

#image handling
import os
import base64
from pathlib import Path
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import fitz  # PyMuPDF
import io


ModuleNotFoundError: No module named 'torch'

In [43]:
#Anthropic key - do not publish
key="slongstuffffhereeeeeee(*&^%$^&%$%^^&GHGGHNBbvhdhhA"

order of operations
1 Split the PDF into 4 quadrants
2 Get the metadata from the titleblock in the bottom right quadrant
3 Get the line list from a Claude query into a JSON string
4 Get the valve list from Yolov5
5 Go through each valve in the valve list and get the size and tag number via a png snippet around the valve. Include the coordinates of valve (include an offset for the quadrant). 
6 Get the line number of the valve, if possible, and add it to the valve list.
7 Get an instrument list using the data from Yolov5. Get the tag name/number for each instrument and coordinates.
8 Get an equipment list from a LLM (may need to use Claude)
8 Use one of the LLMs to create a DEXPI file from the metatdata, line list, valve list, instrument list, and equipment list.


In [73]:
#Split P&ID into 4 quadrants

def pdf_to_images(pdf_path, page_number=0, dpi=300):
    """
    Convert a PDF page to a high-quality PIL Image
    
    Args:
        pdf_path (str): Path to the PDF file
        page_number (int): Page number to convert (0-indexed)
        dpi (int): Resolution for the conversion
    
    Returns:
        PIL.Image: The converted page as an image
    """
    doc = fitz.open(pdf_path)
    
    if page_number >= len(doc):
        raise ValueError(f"Page {page_number} doesn't exist. PDF has {len(doc)} pages.")
    
    page = doc[page_number]
    
    # Create a transformation matrix for higher resolution
    mat = fitz.Matrix(dpi/72, dpi/72)  # 72 is the default DPI
    
    # Render the page as a pixmap
    pix = page.get_pixmap(matrix=mat)
    
    # Convert to PIL Image
    img_data = pix.tobytes("png")
    image = Image.open(io.BytesIO(img_data))
    
    doc.close()
    return image

def split_image_into_quadrants(image):
    """
    Split a PIL Image into 4 equal quadrants
    
    Args:
        image (PIL.Image): The source image
    
    Returns:
        tuple: Four PIL Images representing the quadrants (top-left, top-right, bottom-left, bottom-right)
    """
    width, height = image.size
    
    # Calculate quadrant dimensions
    half_width = width // 2
    half_height = height // 2
    
    # Define crop boxes for each quadrant
    # (left, top, right, bottom)
    top_left = (0, 0, half_width, half_height)
    top_right = (half_width, 0, width, half_height)
    bottom_left = (0, half_height, half_width, height)
    bottom_right = (half_width, half_height, width, height)
    
    # Crop the image into quadrants
    quadrants = (
        image.crop(top_left),
        image.crop(top_right),
        image.crop(bottom_left),
        image.crop(bottom_right)
    )
    
    return quadrants

def save_quadrants_as_png(quadrants, output_dir, base_filename):
    """
    Save quadrant images as PNG files
    
    Args:
        quadrants (tuple): Four PIL Images
        output_dir (str): Directory to save the images
        base_filename (str): Base name for the output files
    
    Returns:
        list: Paths to the saved PNG files
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    quadrant_names = ["top_left", "top_right", "bottom_left", "bottom_right"]
    saved_paths = []
    
    for i, (quadrant, name) in enumerate(zip(quadrants, quadrant_names)):
        filename = f"{base_filename}_q{i+1}.png"
#        filename = f"{base_filename}_quadrant_{i+1}_{name}.png"
        filepath = output_dir / filename
        
        # Save as PNG with high quality
        quadrant.save(filepath, "PNG", optimize=True)
        saved_paths.append(str(filepath))
#        print(f"Saved quadrant {i+1} ({name}): {filepath}")
    
    return saved_paths
    
def create_quads(pdf_path,page,output_dir):
     image = pdf_to_images(pdf_path, page)
     quadrants = split_image_into_quadrants(image)
# Save quadrants as PNG files
     base_filename = Path(pdf_path).stem
     quadrant_paths = save_quadrants_as_png(quadrants, output_dir, base_filename)


In [103]:
# call Ollama LLMs. Encode PNGs too.
def encode_png_file(png_path: str) -> Optional[str]:
    """
    Encode a PNG file to base64
    
    Args:
        png_path (str): Path to the PNG file
    
    Returns:
        str: Base64 encoded image data, or None if error
    """
    try:
        with open(png_path, 'rb') as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"Error encoding {png_path}: {e}")
        return None

def find_png_files(directory: str, pattern: str = "*.png") -> List[str]:
    """
    Find all PNG files in a directory
    
    Args:
        directory (str): Directory to search
        pattern (str): File pattern (default: "*.png")
    
    Returns:
        List[str]: List of PNG file paths
    """
    search_path = os.path.join(directory, pattern)
    png_files = glob.glob(search_path)
    png_files.sort()  # Sort for consistent ordering
    return png_files

def query_ollama(model, prompt, png_path):
    if png_path != "": 
        encoded_image = encode_png_file(png_path)
    
        response = ollama.chat(
           model=model,
           messages=[{'role': 'user', 'content': prompt,'images': [encoded_image]}],
           options={
               'temperature': 0.7,  # Range: 0.0 to 1.0
           }
        )
    else:
        response = ollama.chat(
           model=model,
           messages=[{'role': 'user', 'content': prompt}],
           options={
               'temperature': 0.7,  # Range: 0.0 to 1.0
           }
        )

    foo = response.message.content[8:-4]
    print(response.message.content)
    response_J = json.loads(foo)
    return response_J

def query_ollama_multiple(model, prompt, png_path): #same as above, except png_path is a list of pngs
    #!!!!!This does not work. Right now ollama does not support multiple images
    encoded_images = []
    
    for i,png in enumerate(png_path):
       encoded_images.append(encode_png_file(png))
   
    response = ollama.chat(
       model=model,
       messages=[{'role': 'user', 'content': prompt,'images': [encoded_images]}],
       options={
           'temperature': 0.7,  # Range: 0.0 to 1.0
       }
    )

    foo = response.message.content[8:-4]
    response_J = json.loads(foo)
    return response_J    

In [54]:
#Claude
Claude = "claude-sonnet-4-20250514"

#initialize
client = anthropic.Anthropic(
    api_key=key  # or set ANTHROPIC_API_KEY environment variable
)

def claude_simple(prompt_text):
    content = []
    content.append({
        "type": "text",
        "text": prompt_text
    })
    
    message = client.messages.create(
        model=Claude,
        max_tokens=2000,
        temperature = 0.5,
        messages=[{
            "role": "user",
            "content": content
        }]
    )
    return message.content[0].text 

def upload_multiple_images_to_claude(image_paths, prompt_text):
    """
    Upload multiple images to Claude
    
    Args:
        image_paths (list): List of paths to image files
        prompt_text (str): Text prompt to send along with the images
    
    Returns:
        str: Claude's response
    """
    
    content = []
    
    # Add all images
    for i, image_path in enumerate(image_paths):
        image_path = Path(image_path)
        
        with open(image_path, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode('utf-8')
        
        content.append({
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": image_data
            }
        })
    
    # Add text prompt
    content.append({
        "type": "text",
        "text": prompt_text
    })
    
    message = client.messages.create(
        model=Claude,
        max_tokens=2000,
        temperature = 0.5,
        messages=[{
            "role": "user",
            "content": content
        }]
    )
    
    return message.content[0].text

In [ ]:
# Cropping YOLO predictions
def crop_predictions(png_file_path, predictions_csv_path, scaling_factor=1.0, output_dir="cropped_images"):
    """
    Crop image regions based on predictions CSV with bounding box coordinates.
    
    Parameters:
    png_file_path (str): Path to the input PNG image
    predictions_csv_path (str): Path to the predictions CSV file
    scaling_factor (float): Factor to scale the bounding box size (1.0 = original size)
    output_dir (str): Directory to save cropped images
    """
    
    # Read the CSV file
    df = pd.read_csv(predictions_csv_path)
    
    # Load the image
    image = Image.open(png_file_path)
    image_width, image_height = image.size
    
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Get the base name of the PNG file (without extension)
    base_name = Path(png_file_path).stem
    
    cropped_images = []
    
    for idx, row in df.iterrows():
        # Parse the xyxy coordinates from string representation
        # Remove 'tensor()' wrapper and convert to float
        coords_str = row['Xyxy'].strip('[]')
        coords_parts = coords_str.split(', ')
        
        # Extract numerical values from tensor format
        coords = []
        for part in coords_parts:
            # Remove 'tensor(' and ')' and convert to float
            num_str = part.strip().replace('tensor(', '').replace(')', '').replace('.', '')
            coords.append(float(num_str))
        
        x1, y1, x2, y2 = coords
        
        # Calculate center and dimensions of bounding box
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        width = x2 - x1
        height = y2 - y1
        
        # Apply scaling factor
        scaled_width = width * scaling_factor
        scaled_height = height * scaling_factor
        
        # Calculate new coordinates
        new_x1 = center_x - scaled_width / 2
        new_y1 = center_y - scaled_height / 2
        new_x2 = center_x + scaled_width / 2
        new_y2 = center_y + scaled_height / 2
        
        # Ensure coordinates are within image bounds
        new_x1 = max(0, min(new_x1, image_width))
        new_y1 = max(0, min(new_y1, image_height))
        new_x2 = max(0, min(new_x2, image_width))
        new_y2 = max(0, min(new_y2, image_height))
        
        # Crop the image
        cropped_img = image.crop((new_x1, new_y1, new_x2, new_y2))
        
        # Create filename for cropped image
        prediction_clean = row['Prediction'].replace('/', '_').replace(' ', '_')
        confidence = f"{row['Confidence']:.2f}"
        output_filename = f"{base_name}_{idx:03d}_{prediction_clean}_conf{confidence}.png"
        output_path = os.path.join(output_dir, output_filename)
        
        # Save cropped image
        cropped_img.save(output_path)
        
        cropped_images.append({
            'index': idx,
            'prediction': row['Prediction'],
            'confidence': row['Confidence'],
            'original_coords': (x1, y1, x2, y2),
            'scaled_coords': (new_x1, new_y1, new_x2, new_y2),
            'output_path': output_path,
            'size': cropped_img.size
        })
        
        print(f"Saved: {output_filename} ({cropped_img.size[0]}x{cropped_img.size[1]})")
    
    print(f"\nProcessed {len(cropped_images)} predictions")
    print(f"Cropped images saved to: {output_dir}")
    
    return cropped_images


In [ ]:
# Valve crop and look up 
def lookup_valve(valves, png_file_path, predictions_csv_path, scaling_factor=1.0, output_dir="cropped_images"):
    
    # Read the CSV file
    df = pd.read_csv(predictions_csv_path)
    
    # Load the image
    image = Image.open(png_file_path)
    image_width, image_height = image.size
    
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Get the base name of the PNG file (without extension)
    base_name = Path(png_file_path).stem
    
    cropped_images = []
    
    for idx, row in df.iterrows():
        # Parse the xyxy coordinates from string representation
        # Remove 'tensor()' wrapper and convert to float
        coords_str = row['Xyxy'].strip('[]')
        coords_parts = coords_str.split(', ')
        prediction = row['Prediction']
        
        
        # Extract numerical values from tensor format
        coords = []
        for part in coords_parts:
            # Remove 'tensor(' and ')' and convert to float
            num_str = part.strip().replace('tensor(', '').replace(')', '').replace('.', '')
            coords.append(float(num_str))
        
        x1, y1, x2, y2 = coords
        
        # Calculate center and dimensions of bounding box
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        width = x2 - x1
        height = y2 - y1
        
        # Apply scaling factor
        scaled_width = width * scaling_factor
        scaled_height = height * scaling_factor
        
        # Calculate new coordinates
        new_x1 = center_x - scaled_width / 2
        new_y1 = center_y - scaled_height / 2
        new_x2 = center_x + scaled_width / 2
        new_y2 = center_y + scaled_height / 2
        
        # Ensure coordinates are within image bounds
        new_x1 = max(0, min(new_x1, image_width))
        new_y1 = max(0, min(new_y1, image_height))
        new_x2 = max(0, min(new_x2, image_width))
        new_y2 = max(0, min(new_y2, image_height))
        
        # Crop the image
        cropped_img = image.crop((new_x1, new_y1, new_x2, new_y2))
        
        # Create filename for cropped image
        prediction_clean = row['Prediction'].replace('/', '_').replace(' ', '_')
        confidence = f"{row['Confidence']:.2f}"
        output_filename = f"{base_name}_{idx:03d}_{prediction_clean}_conf{confidence}.png"
        output_path = os.path.join(output_dir, output_filename)
        
        # Save cropped image
        cropped_img.save(output_path)
        
        cropped_images.append({
            'index': idx,
            'prediction': row['Prediction'],
            'confidence': row['Confidence'],
            'original_coords': (x1, y1, x2, y2),
            'scaled_coords': (new_x1, new_y1, new_x2, new_y2),
            'output_path': output_path,
            'size': cropped_img.size
        })
        
        print(f"Saved: {output_filename} ({cropped_img.size[0]}x{cropped_img.size[1]})")
    
    print(f"\nProcessed {len(cropped_images)} predictions")
    print(f"Cropped images saved to: {output_dir}")
    
    return cropped_images


In [ ]:
#main
pdf_file = "/home/peter/test3.pdf"
page = 0
ollama_model = 'gemma3:27b'

output_dir = pdf_file[:-4]
filename = os.path.splitext(os.path.basename(pdf_file))[0]
create_quads(pdf_file,page,output_dir) #create PNG files

# Call Gemma3:27b via Ollama to get metadata
prompt = """List the project name, company name, drawing (DWG) name, drawing number, location, date, revision, author, 
and any other information found in the title block in the lower right corner of the image. Return data as JSON."""
png_path = output_dir+"/"+filename+"_q4.png"
drawing_info = query_ollama(ollama_model, prompt, png_path
                           )
# Call Claude and get line list. Need to send all 4 quadrants
full_dwg = [] #all 4 quadrants
for i in range(4):
    n = i+1
    full_dwg.append(output_dir+"/"+filename+"_q"+str(n)+".png")
prompt = """list the pipe lines on this P&ID with the line number, primary size, and what equipment, off-page connectors, 
or other lines it is connected to. Return the data as JSON and only return JSON."""
result = upload_multiple_images_to_claude(full_dwg, prompt)
line_list = json.loads(result[8:-4])

#equipment list
prompt = """List the equipment on this P&ID with the name, description and associated metadata. Only list the primary equipment. 
Do not list piping, valves, instruments or information about the drawing itself. Return data as JSON"""
result = upload_multiple_images_to_claude(full_dwg, prompt)
equip_list = json.loads(result[8:-4])

#valves
valves = []
#for each quadrant
for quad in range(4):
    #clean up previous runs
    directory_path = "/home/peter/Desktop/yolov5/runs/detect"
    shutil.rmtree(directory_path)
    #image size is important - if the size is too far off, detection is not accurate
    !python "./detect.py" --weights "./best.pt" --source full_dwg[quad] --conf-thres 0.5 --save-csv --imgsz 5000
    result_path = directory_path+"/exp"
    #create cropped images
    crop_predictions(full_dwg[quad], result_path, scaling_factor=10.0, output_dir=results_path)
    #go through each detected item and add the ones we want to the valve list
    


In [ ]:
%cd /home/peter/Desktop/yolov5
!python3 --version
!source .venv/bin/activate
!python3 "detect.py" --weights "./best.pt" --source "/home/peter/test3/test3_q1.png"

In [ ]:
#remember to clean up png files when done.